In [121]:
#library to scrape the web
#import scrapy
from bs4 import BeautifulSoup

import requests
from scrapy.http import TextResponse

import json
import pandas as pd
import time


In [117]:
#master job listing list
link_list=[]

#invidual job details list
job_details_list=[]

#run scrapper while flag is True
run_flag=True
page = 1
job_listing_no = 0
while(run_flag):
    link_list=[]
    job_details_list=[]
    #read json page
    time.sleep(1)
    url = 'http://service.dice.com/api/rest/jobsearch/v1/simple.json?text=data+scientist&page=' + str(page)
    r = requests.get(url)
    page_data = json.loads(r.text)
    
    #read the element with all the job url info
    link_data = page_data['resultItemList']
    
    print('processing page ' + str(page))
    print('jobs so far ' + str(job_listing_no))
    
    #read links on page
    for link in link_data:
        job_listing_no += 1
        link_dict = {}
        for ky in link.keys():
            link_dict[ky] = link[ky]
        link_dict['page'] = page
        link_dict['job_listing_no'] = job_listing_no
        link_list.append(link_dict)
        
        #going to specific 
        time.sleep(1)
        job_url = link_dict['detailUrl']
        r_job = requests.get(job_url)
        
        #using bs4 to read 
        soup = BeautifulSoup(r_job.text, 'html.parser')
        
        #get only job-info tags class
        job_info_list = soup.findAll('div',{'class':'row job-info'})
        
        
        
        job_details = {}
        for i in job_info_list:
            #print(job_details)
            #print(i)
            #find technical skills
            if(i.find('span',{'class':'icon-plugin-1'})):
                job_spec = i.find('div',{'class':'iconsiblings'}).text.strip()
                job_details['technical_skills'] = job_spec
                #print(job_spec)
            else:
                if(job_details.get('technical_skills','not_found')=='not_found'):
                    job_details['technical_skills'] = ''

            if(i.find('span',{'class':'icon-briefcase'})):
                job_spec = i.find('div',{'class':'iconsiblings'}).text.strip()
                job_details['job_type'] = job_spec
                #print(job_spec)
            else:
                if(job_details.get('job_type','not_found')=='not_found'):
                    job_details['job_type'] = ''

            if(i.find('span',{'class':'icon-bank-note'})):
                job_spec = i.find('div',{'class':'iconsiblings'}).text.strip()
                job_details['salary_info'] = job_spec
                #print(job_spec)
            else:
                if(job_details.get('salary_info','not_found')=='not_found'):
                    job_details['salary_info'] = ''

            if(i.find('span',{'class':'icon-network-2'})):
                job_spec = i.find('div',{'class':'iconsiblings'}).text.strip()
                job_details['travel_info'] = job_spec
                #print(job_spec)
            else:
                if(job_details.get('travel_info','not_found')=='not_found'):
                    job_details['travel_info'] = ''
            #break
        job_details['page'] = page
        job_details['job_listing_no'] = job_listing_no
        job_details_list.append(job_details)
    
    page_job_data = pd.DataFrame(link_list)
    page_job_data.to_csv('data/page_job_data'+ str(page) + '.csv',index=False)
    
    page_job_detail_data = pd.DataFrame(job_details_list)
    page_job_detail_data.to_csv('data/page_job_detail_data'+ str(page) + '.csv',index=False)
    #incrementing page
    page += 1
    
    if job_listing_no >= 30087:
        run_flag=False




processing page 1
jobs so far 0
processing page 2
jobs so far 50
processing page 3
jobs so far 100
processing page 4
jobs so far 150
processing page 5
jobs so far 200
processing page 6
jobs so far 250
processing page 7
jobs so far 300
processing page 8
jobs so far 350
processing page 9
jobs so far 400
processing page 10
jobs so far 450
processing page 11
jobs so far 500
processing page 12
jobs so far 550
processing page 13
jobs so far 600
processing page 14
jobs so far 650
processing page 15
jobs so far 700
processing page 16
jobs so far 750
processing page 17
jobs so far 800
processing page 18
jobs so far 850
processing page 19
jobs so far 900
processing page 20
jobs so far 950
processing page 21
jobs so far 1000
processing page 22
jobs so far 1050
processing page 23
jobs so far 1100
processing page 24
jobs so far 1150
processing page 25
jobs so far 1200
processing page 26
jobs so far 1250
processing page 27
jobs so far 1300
processing page 28
jobs so far 1350
processing page 29
jobs 

In [111]:
job_listing_no == 30087:
    run_flag=False

100

In [120]:
#pages pulled
print(page)


#job listing pulled
print(job_listing_no)

603
30100


In [118]:
#pd.DataFrame(link_list)

In [100]:
pd.DataFrame(job_details_list)

,job_type,salary_info,technical_skills,travel_info
0,"Full Time, Contract Corp-To-Corp, Contract Ind...",150-200K+RSU+BONUS,"R, Python, JavaScript, C, Java, SQL, Data Sci...",Telecommuting not available\nTravel not required


In [93]:
[str.lower(skl.strip()) for skl in job_details['technical_skills'].split(',')]

['r',
 'python',
 'javascript',
 'c',
 'java',
 'sql',
 'data scientist',
 'machine learning',
 'data engineer',
 'hadoop',
 'hive',
 'snowflake',
 'pig',
 'scala']

In [124]:
page_ctr=1
job_sum = pd.read_csv('data/page_job_data'+ str(page_ctr) + '.csv')

In [125]:
job_sum.head()

,company,date,detailUrl,jobTitle,job_listing_no,location,page
0,"Projas Technologies, LLC",2017-04-18,http://www.dice.com/job/result/10126850/DATA-E...,Senior Data Engineer *** 200K or contract to h...,1,"San Francisco, CA",1
1,Creative Data Resources,2017-04-05,http://www.dice.com/job/result/cdatar/933219?s...,Director of Data Science,2,"New York, NY",1
2,Fusion Plus Solutions Inc,2017-04-19,http://www.dice.com/job/result/10529307/963370...,Data Scientist or data analyst in PA,3,"Philadelphia, PA",1
3,UnitedHealth Group,2017-04-22,http://www.dice.com/job/result/uhgbot/704376?s...,Senior Data Modeling Analyst (Data Scientist) ...,4,"Minnetonka, MN",1
4,Zillow.com,2017-04-22,http://www.dice.com/job/result/RTX156587/oTmq3...,"Sr. Principal Data Scientist, Machine Learning",5,"Seattle, WA",1


In [128]:
try:
    page_ctr=1
    while(True):
        if page_ctr==1:
            job_sum = pd.read_csv('data/page_job_data'+ str(page_ctr) + '.csv')
        else:
            temp_sum = pd.read_csv('data/page_job_data'+ str(page_ctr) + '.csv')
            job_sum = pd.concat([job_sum,temp_sum])
            
        page_ctr += 1
except:
    print('pages looped ' + str(page_ctr))
    print('looped through all files')
        

pages looped 603
looped through all files


In [129]:
job_sum.shape

(30100, 7)

In [130]:
job_sum.to_csv('data/job_listing_header.csv',index=False)

In [131]:
try:
    page_ctr=1
    while(True):
        if page_ctr==1:
            job_det = pd.read_csv('data/page_job_detail_data'+ str(page_ctr) + '.csv')
        else:
            temp_det = pd.read_csv('data/page_job_detail_data'+ str(page_ctr) + '.csv')
            job_det = pd.concat([job_det,temp_det])
            
        page_ctr += 1
except:
    print('pages looped ' + str(page_ctr))
    print('looped through all files')

pages looped 603
looped through all files


In [132]:
job_det.to_csv('data/job_listing_detail.csv',index=False)